# <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">AB Testing Comparison</div>

### Business Problem
Baru-baru ini, Facebook memperkenalkan opsi penawaran baru yang disebut "average bidding" sebagai alternatif terhadap metode "maximum bidding" yang sudah ada. Salah satu klien yaitu, bombabomba.com, telah memutuskan untuk menguji fitur baru ini untuk menentukan apakah "average bidding" menghasilkan lebih banyak konversi daripada metode "maximum bidding". Mereka telah menjalankan uji A/B selama sebulan dan sekarang kita akan menganalisis hasil uji A/B ini. Metrik keberhasilan utama untuk bombabomba.com adalah Pembelian. Oleh karena itu, uji statistik harus berfokus pada metrik Pembelian.

### Dataset Story
Dataset ini berisi informasi tentang situs web sebuah perusahaan, termasuk jumlah iklan yang dilihat dan diklik oleh pengguna, serta pendapatan yang dihasilkan dari tindakan-tindakan tersebut. Ada dua set data terpisah untuk kelompok Control dan Test, yang terletak di lembar yang berbeda dari file Excel "ab_testing.xlsx". Kelompok Control telah diuji dengan Maximum Bidding, sedangkan kelompok Test telah diuji dengan Average Bidding.

- **Impression:** Jumlah tampilan iklan
- **Click:** Jumlah klik pada iklan yang ditampilkan
- **Purchase:** Jumlah produk yang dibeli setelah mengklik iklan
- **Earning:** Pendapatan yang dihasilkan dari produk yang dibeli

## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Libraries Import</div>

In [36]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Persiapan dan Analisis Data</div>

In [37]:
xlsx = pd.ExcelFile('/kaggle/input/ab-test12/ab_testing.xlsx')
df_c = pd.read_excel(xlsx, "Control Group")
df_t = pd.read_excel(xlsx, "Test Group")

### Detail Dataset Control Group

In [38]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [39]:
print("Missing Value:")
df_c.isnull().sum()

Missing Value:


Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [40]:
df_c.describe()

,Impression,Click,Purchase,Earning
count,40.00000,40.00000,40.00000,40.00000
mean,101711.44907,5100.65737,550.89406,1908.56830
std,20302.15786,1329.98550,134.10820,302.91778
min,45475.94296,2189.75316,267.02894,1253.98952
25%,85726.69035,4124.30413,470.09553,1685.84720
50%,99790.70108,5001.22060,531.20631,1975.16052
75%,115212.81654,5923.80360,637.95709,2119.80278
max,147539.33633,7959.12507,801.79502,2497.29522


### Detail Dataset Test Group

In [41]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [42]:
print("Missing Value:")
df_t.isnull().sum()

Missing Value:


Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [43]:
df_t.describe()

,Impression,Click,Purchase,Earning
count,40.00000,40.00000,40.00000,40.00000
mean,120512.41176,3967.54976,582.10610,2514.89073
std,18807.44871,923.09507,161.15251,282.73085
min,79033.83492,1836.62986,311.62952,1939.61124
25%,112691.97077,3376.81902,444.62683,2280.53743
50%,119291.30077,3931.35980,551.35573,2544.66611
75%,132050.57893,4660.49791,699.86236,2761.54540
max,158605.92048,6019.69508,889.91046,3171.48971


## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Penggabungan Dataset</div>

In [44]:
# Gabungkan dataset Control dan Test setelah analisis menggunakan metode concat pandas
df = pd.concat([df_t, df_c])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  80 non-null     float64
 1   Click       80 non-null     float64
 2   Purchase    80 non-null     float64
 3   Earning     80 non-null     float64
dtypes: float64(4)
memory usage: 3.1 KB


## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Hipotesis A/B Testing</div>

**H0**: M1 = M2
**H1**: M2 != M2

Dalam konteks A/B testing, mendefinisikan hipotesis adalah langkah penting yang menentukan tahap untuk menguji efek dua perlakuan atau kondisi yang berbeda terhadap hasil yang diminati. Misalnya, ketika membandingkan dua strategi penawaran yang berbeda seperti "maximum bidding" (Control group) dan "average bidding" (Test group), kita bertujuan untuk memahami apakah ada perbedaan yang signifikan secara statistik dalam dampaknya terhadap hasil pembelian.

### Hipotesis
- **Hipotesis Nol (H0)**: Hipotesis nol menyatakan bahwa tidak ada perbedaan dalam nilai rata-rata pembelian antara kelompok Control dan kelompok Test. Secara matematis, dapat direpresentasikan sebagai \(H_0: M_1 = M_2\), dimana \(M_1\) dan \(M_2\) adalah nilai rata-rata pembelian untuk kelompok Control dan kelompok Test, berturut-turut.

- **Hipotesis Alternatif (H1)**: Hipotesis alternatif menyarankan bahwa ada perbedaan dalam nilai rata-rata pembelian antara kedua kelompok. Ini direpresentasikan sebagai \(H_1: M_1 \neq M_2\). Hipotesis ini adalah apa yang diuji untuk dibuktikan atau didukung oleh tes.

### Menganalisis Rata-rata Pembelian untuk Kelompok Control dan Test
Untuk memahami potensi dampak dari kedua strategi penawaran, dilakukan analisis awal terhadap rata-rata pembelian untuk kedua kelompok Control dan Test. Ini melibatkan perhitungan nilai rata-rata pembelian untuk setiap kelompok, yang memberikan wawasan awal apakah mungkin ada perbedaan yang signifikan antara dua strategi penawaran tersebut.
- **Rata-rata pembelian kelompok Control**: rata-rata nilai pembelian dalam kelompok Control, yang dikenakan strategi Maximum Bidding.

- **Rata-rata pembelian kelompok Test**: rata-rata nilai pembelian dalam kelompok Test, yang dikenakan strategi Average Bidding.

Analisis awal ini penting untuk menyiapkan A/B test dan memutuskan uji statistik yang akan digunakan untuk pengujian hipotesis. Hal ini dapat membantu dalam memahami arah dan besarnya perbedaan. Jika ada, antara kelompok Control dan Test sebelum melakukan analisis statistik yang lebih rinci.

In [45]:
df_list = [df_c, df_t]
for roi_df in df_list:
    print(roi_df["Purchase"].mean())

550.8940587702316
582.1060966484677


## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Penerapan Pengujian Hipotesis</div>

1. Menetapkan Hipotesis
2. Pengecekan Asumsi
> 2.1. Asumsi Normalitas (shapiro) <br> 2.2. Homogenitas Varians (levene)
3. Penerapan Hipotesis
> 3.1 Uji t dua sampel independen jika asumsi terpenuhi <br> 3.2 Uji Mann-Whitney jika asumsi tidak terpenuhi
4. Menginterpretasikan hasil berdasarkan nilai p (p-value), Note:
- Jika normalitas tidak tercapai, langsung masuk ke langkah nomor 2. Jika homogenitas varians tidak tercapai, masuk ke langkah nomor 1.
- Mungkin berguna untuk melakukan tinjauan dan koreksi outlier sebelum meninjau normalitas.

### A) Uji Asumsi Normalitas

**H0**: Asumsi distribusi normal terpenuhi.  
**H1**: ..tidak disediakan.

Asumsi normalitas adalah pertimbangan penting dalam statistik, khususnya saat menerapkan uji parametrik yang mengharuskan data mengikuti distribusi normal. Asumsi ini beranggapan bahwa titik data dalam suatu himpunan data terdistribusi normal di sekitar mean. Pentingnya asumsi ini terletak pada dampaknya terhadap keakuratan dan keandalan inferensi statistik.

#### Mengapa Asumsi Normalitas Penting?

- **Landasan untuk Uji Parametrik**: Banyak uji statistik, termasuk uji-t dan ANOVA, bergantung pada asumsi normalitas. Uji-uji ini mengasumsikan bahwa data, atau residual dalam analisis regresi, terdistribusi normal untuk secara akurat memperkirakan parameter yang diminati dan untuk menghitung statistik uji.

- **Kepercayaan pada Hasil**: Mematuhi asumsi normalitas meningkatkan kepercayaan pada hasil analisis statistik. Ketika data terdistribusi normal, estimasi mean, varians, dan interval kepercayaan lebih cenderung tidak bias dan mencerminkan parameter populasi yang sebenarnya.

- **Mengurangi Kesalahan Tipe I dan Tipe II**: Memenuhi asumsi normalitas membantu meminimalkan kesalahan Tipe I (menolak hipotesis nol yang benar secara salah) dan kesalahan Tipe II (gagal menolak hipotesis nol yang salah). Ini meningkatkan daya uji, yang merupakan probabilitas dalam menolak hipotesis nol yang salah dengan benar.

#### Cara Menguji Normalitas:

Beberapa metode tersedia untuk menguji asumsi normalitas, termasuk metode grafis dan uji statistik:

- **Metode Grafik**: Plot seperti histogram, plot Q-Q (quantile-quantile), dan plot P-P (probability-probability) secara visual menilai distribusi data. Penyimpangan dari linearitas dalam plot Q-Q dan P-P menunjukkan penyimpangan dari normalitas.

- **Uji Statistik**: Uji Shapiro-Wilk dan uji Kolmogorov-Smirnov termasuk di antara uji statistik yang paling banyak digunakan untuk menilai normalitas. Uji-uji ini membandingkan distribusi data dengan distribusi normal dan memberikan p-value untuk memutuskan asumsi normalitas. P-value yang tinggi (biasanya > 0,05) menunjukkan bahwa data tidak secara signifikan menyimpang dari normalitas.

Sebagai kesimpulan, Uji Asumsi Normalitas adalah langkah penting dalam mempersiapkan data untuk analisis statistik. Ini memastikan kelayakan uji parametrik dan berkontribusi pada keandalan dan validitas hasil uji.

In [46]:
test_stat, pvalue = shapiro(df_c["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


**H0** ditolak jika p-value < 0.05, maka bukti yang diamati cukup kuat untuk menolak hipotesis nol. Dengan kata lain, data cukup menunjukkan adanya perbedaan atau efek yang tidak dapat dijelaskan oleh kebetulan saja.

Jika p-value tidak < 0.05 maka **H0** TIDAK DAPAT DITOLAK. Hal ini berarti tidak ada bukti yang cukup untuk menolak hipotesis nol. Ini tidak berarti hipotesis nol benar, tetapi hanya saja data tidak memberikan bukti yang kuat untuk menolaknya.

In [47]:
test_stat, pvalue = shapiro(df_t["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


### B) Uji Homogenitas Variansi

**H0**: Varian Homogen.
**H1**: Varian tidak Homogen.

Asumsi homogenitas variansi (HoV) adalah prasyarat fundamental untuk uji statistik tertentu, seperti ANOVA dan uji-t sampel independen. Asumsi ini menyatakan bahwa semua kelompok perbandingan memiliki variansi yang sama pada variabel dependen. Alasan di balik asumsi ini adalah untuk memastikan perbandingan yang adil dan akurat antar kelompok.

#### Mengapa Asumsi Homogenitas Variansi Penting?

- **Varians Error yang Sama**: Uji statistik yang mengandalkan asumsi ini menganggap bahwa varians error, atau penyebaran titik data di sekitar mean dalam kelompok yang berbeda, adalah sama. Keseragaman ini memungkinkan perbandingan mean antar kelompok didasarkan pada landasan yang sama, memastikan bahwa perbedaan yang terdeteksi dikaitkan dengan variabel independen daripada varians yang tidak sama.

- **Akurasi Uji**: Memenuhi asumsi HoV membantu menjaga akurasi tingkat kesalahan Tipe I uji. Ketika varians tidak sama, risiko menolak hipotesis nol secara salah (kesalahan Tipe I) dapat meningkat, yang mengarah pada kesimpulan yang salah.

- **Daya Statistik**: Mematuhi asumsi ini juga dapat mempengaruhi daya statistik uji, yang merupakan kemampuan untuk menolak hipotesis nol dengan benar ketika salah. Varian yang tidak sama dapat menyebabkan hilangnya daya, sehingga lebih sulit untuk mendeteksi perbedaan nyata ketika ada.

#### Cara Menguji Homogenitas Variansi:

Berbagai uji statistik memeriksa asumsi HoV, dengan uji Levene sebagai salah satu yang paling umum digunakan. Uji Levene menilai apakah varians antar kelompok secara statistik berbeda secara signifikan. Jika uji menunjukkan tidak ada perbedaan signifikan dalam varians, asumsi homogenitas varians dianggap terpenuhi, memungkinkan penggunaan uji parametrik yang mengandalkan asumsi ini.

Singkatnya, menguji homogenitas variansi merupakan langkah penting dalam persiapan data untuk analisis. Ini memastikan bahwa kondisi untuk uji statistik tertentu terpenuhi, sehingga menjaga validitas dan keandalan hasil uji.

In [48]:
test_stat, pvalue = levene(df_c["Purchase"],
                           df_t["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


**H0** ditolak jika p-value < 0.05, maka bukti yang diamati cukup kuat untuk menolak hipotesis nol. Dengan kata lain, data cukup menunjukkan adanya perbedaan atau efek yang tidak dapat dijelaskan oleh kebetulan saja.

Jika p-value tidak < 0.05 maka **H0** TIDAK DAPAT DITOLAK. Hal ini berarti tidak ada bukti yang cukup untuk menolak hipotesis nol. Ini tidak berarti hipotesis nol benar, tetapi hanya saja data tidak memberikan bukti yang kuat untuk menolaknya.

## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Penerapan Hipotesis</div>

### Melakukan Uji t-dua sampel independen

Setelah menentukan hipotesis untuk pengujian A/B dan menganalisis rata-rata pembelian untuk kedua kelompok Control (Maximum Bidding) dan Test (Average Bidding), melanjutkan ke fase analisis statistik. Uji t-dua sampel independen digunakan untuk membandingkan nilai pembelian rata-rata antara kelompok ini, dan inilah alasan mengapa uji ini dipilih:

#### Mengapa Menggunakan Uji t-Dua Sampel Independen?

- **Kesejajaran dengan Tujuan**: Uji t-dua sampel independen secara khusus dirancang untuk membandingkan mean antara dua kelompok independen. Ini menjadikannya pilihan ideal untuk tujuan ini, yaitu untuk menilai apakah terdapat perbedaan signifikan secara statistik dalam nilai pembelian antara kelompok Control dan Test.

- **Independensi Sampel**: Uji ini mengasumsikan bahwa titik data dalam kedua kelompok saling independen. Mengingat bahwa kelompok ini mewakili strategi penawaran yang berbeda yang diterapkan pada segmen pengguna yang berbeda, asumsi ini valid.

- **Asumsi Variansi Sama**: Dengan menggunakan uji dengan `equal_var=True`, melanjutkan dengan asumsi bahwa varians jumlah pembelian serupa di kedua kelompok. Asumsi ini idealnya didukung oleh uji homogenitas variansi sebelumnya (misalnya, uji Levene).

In [49]:
test_stat, pvalue = ttest_ind(df_c["Purchase"],
                              df_t["Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


## <div style="background-color:#153B50;font-family:newtimeroman;color:#FFFFFF;font-size:150%;text-align:center;border-radius:10px 10px;">Analisis Hasil</div>

### Uji yang Digunakan dan Alasannya

Uji t-dua sampel independen (ttest_ind) digunakan. Uji ini menentukan apakah perbedaan rata-rata antara dua kelompok independen signifikan secara statistik. Alasan utama pemilihan uji ini meliputi:

- **Asumsi Normalitas**: Asumsi normalitas diperiksa menggunakan uji Shapiro-Wilk untuk variabel 'Purchase' di kedua kelompok. Tergantung apakah asumsi ini terpenuhi, uji parametrik seperti t-test cocok untuk data terdistribusi normal.

- **Homogenitas Variansi**: Homogenitas varians diperiksa menggunakan uji Levene untuk variabel 'Purchase' di kedua kelompok. Homogenitas varians merupakan asumsi penting untuk uji-t.

Dalam kondisi ini, jika asumsi terpenuhi (atau tidak terpenuhi tetapi disesuaikan dengan tepat), uji t-dua sampel independen dapat diterapkan.

### Hasil Uji dan Rekomendasi untuk Klien

Hasil uji, Statistik Uji = -0.9416, p-value = 0.3493, menunjukkan bahwa tidak ada cukup bukti untuk menolak hipotesis nol. Ini menunjukkan bahwa tidak ada perbedaan signifikan secara statistik dalam 'Purchase' rata-rata antara kelompok Control (Maximum Bidding) dan kelompok Test (Average Bidding).

### Rekomendasi:

Mengingat p-value 0,3493, yang melebihi tingkat alpha standar 0,05, disimpulkan bahwa tidak ada perbedaan signifikan dalam konversi antara strategi Maximum Bidding dan Average Bidding. Dengan demikian, dari sudut pandang statistik, beralih ke Average Bidding tidak serta merta menghasilkan peningkatan pembelian yang signifikan dibandingkan dengan Maximum Bidding.

Namun, pengambilan keputusan juga harus mempertimbangkan faktor lain seperti efisiensi biaya, skalabilitas, dan tujuan strategis jangka panjang. Jika Average Bidding menawarkan keuntungan lain yang tidak ditangkap oleh uji ini (misalnya, jangkauan yang lebih baik, penargetan audiens yang lebih baik), mungkin masih perlu dipertimbangkan meskipun tidak ada perbedaan signifikan dalam tingkat pembelian.